In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
    
def normalize_points(img):
  """
  input: img(N, 2) N number of points in 1 image

  outputs: img_normalized(N, 3), T(3,3)
  """

  mean, std = np.mean(img, axis=0), np.std(img)

  T = np.array([[std/np.sqrt(2), 0, mean[0]],
                [0, std/np.sqrt(2), mean[1]],
                [0,   0, 1]])

  T_inv = np.linalg.inv(T)
  img_aug = np.column_stack([img, np.ones(img.shape[0])]).T

  img_normalized = T_inv.dot(img_aug).T

  return img_normalized, T_inv



def skewed(a, b, c):
  A = np.zeros((2,9))
  A[0, 3:6] = -c
  A[0, 6:] = b
  A[1, :3] = c
  A[1, 6:] = -a
  return A  

def DLT(pnt_set1, pnt_set2, num_pnts):
  """
  inputs: pnt_set1 (N,3)
  """
  # Create the A(2,9) matrix for each point correspondences
  A = []
  for i in range(num_pnts):
        p1 = pnt_set1[i]
        p2 = pnt_set2[i]
        p1_transpose = p1.T
        c = p2[2] * p1_transpose
        b = p2[1] * p1_transpose
        a = p2[0] * p1_transpose
        A.append(skewed(a, b, c))  # (2, 9)

  A = np.concatenate(A, axis=0)
  U, D, V = np.linalg.svd(A)
  h = V[-1, :] / V[-1, -1]
  H = h.reshape(3,3)
  #H = np.zeros((3,3))
  #H[0, :] = h[:3].T
  #H[1, :] = h[3:6].T
  #H[2, :] = h[6:].T
  
  return H


def calc_homography(src_pnts, target_pnts, num_pnts=4):
  
  # Normalize the points through similarity transform
  norm_pnts1, T1 = normalize_points(src_pnts)
  norm_pnts2, T2 = normalize_points(target_pnts)

  #print(norm_pnts1[0])
  # Apply Direct Linear Transform (DLT)
  normalized_H = DLT(norm_pnts1, norm_pnts2, num_pnts)

  # Denormalize the estimated homography
  H = np.dot(np.dot(np.linalg.pinv(T2), normalized_H), T1)

  return H


def read_images(img1_pth, img2_pth, mask1_pth, mask2_pth):
  img1 = cv2.imread(img1_pth)
  img2 = cv2.imread(img2_pth)

  mask1 = cv2.imread(mask1_pth)
  mask2 = cv2.imread(mask2_pth)
  
  outputs = {"img1": img1, "img2": img2, "mask1": mask1, "mask2": mask2}
  return outputs

def detect_and_match_features(inputs, use_mask=False):
  img1 = inputs["img1"]
  img2 = inputs["img2"]
  mask1 = inputs["mask1"]
  mask2 = inputs["mask2"]

  # Convert images to gray scale format
  gray_img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
  gray_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

  # Detect and Describe features in both images using ORB algorithm
  orb = cv2.ORB_create(nfeatures=10000)
  if use_mask:
        kp1, des1 = orb.detectAndCompute(gray_img1, mask=mask1)
        kp2, des2 = orb.detectAndCompute(gray_img2, mask=mask2)
  else:
        kp1, des1 = orb.detectAndCompute(gray_img1, mask=None)
        kp2, des2 = orb.detectAndCompute(gray_img2, mask=None)

  # Match the features by using Flann method
  index_params = dict(algorithm=6,
                        table_number=6,
                        key_size=12,
                        multi_probe_level=2)
  search_params = {}
  flann = cv2.FlannBasedMatcher(index_params, search_params)
  matches = flann.knnMatch(des1, des2, k=2)

  # According to the Lowe's ratio, test to filter good matches
  good_matches = []
  for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

  src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
  target_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

  return src_points, target_points

def ransac(src_points, target_points, params):
  # Find homography with RANSAC
  num_iter = params["num_iter"]
  epsilon = params["epsilon"]
  threshold = params["threshold"]
  num_pnts = params["num_pnts"]
  
  best_inliers1 = []
  best_inliers2 = []
  index = np.arange(0, src_points.shape[0])
  for i in range(num_iter):
        #np.random.shuffle(index)
        rand_idx = np.random.choice(index, num_pnts, replace=False)
        p1_set = src_points[rand_idx, :] # (num_pnts, 2) np array
        p2_set = target_points[rand_idx, :] # (num_pnts, 2) np array
        H = calc_homography(p1_set, p2_set, num_pnts)
        inliers1 = []
        inliers2 = []
        for i in range(src_points.shape[0]):
          p1 = np.expand_dims(src_points[i, :], 0)
          p2 = np.expand_dims(target_points[i, :], 0)
          p1_aug = np.column_stack([p1, np.ones((1,1))]).T
          p2_aug = np.column_stack([p2, np.ones((1,1))]).T
          distance = (np.linalg.norm(p2_aug - H.dot(p1_aug)))**2
          if distance <= epsilon:
            inliers1.append(p1)
            inliers2.append(p2)
        
        if len(inliers1) > len(best_inliers1):
          best_inliers1 = inliers1
          best_inliers2 = inliers2
          if len(best_inliers1)/src_points.shape[0] > threshold:
                  print("The dominant plane is found!!!")
                  print("The best distance is ", distance)
                  break

  # Calculate final homography with the best inliers
  #print(np.array(best_inliers1).shape)
  inlier_points1 = np.array(best_inliers1).squeeze(1)
  inlier_points2 = np.array(best_inliers2).squeeze(1)
  final_H = calc_homography(inlier_points1, inlier_points2, len(best_inliers1))
  outputs = {"H": final_H, "best_inliers1": best_inliers1, "best_inliers2": best_inliers2}
  return outputs

def panorama_creation(img1, img2, H):
  width = img1.shape[1] + img2.shape[1]
  height = img1.shape[0] + img2.shape[0]

  result = cv2.warpPerspective(img1, H, (width, height))
  result[0:img2.shape[0], 0:img2.shape[1]] = img2

  plt.figure(figsize=(20,10))
  plt.imshow(result)

  plt.axis('off')
  plt.show()
  cv2.imwrite("out3.jpg", result)
def main():

  # Read images
  img_path1 = "/content/drive/MyDrive/3D_Vision_Class/term_project/part1/part1_images4/IMG_4614.JPG"
  img_path2 = "/content/drive/MyDrive/3D_Vision_Class/term_project/part1/part1_images4/IMG_4615.JPG"
  #img_path1 = "/content/drive/MyDrive/3D_Vision_Class/term_project/part1/img56.jpg"
  #img_path2 = "/content/drive/MyDrive/3D_Vision_Class/term_project/part1/img57.jpg"


  # Read masks
  img_mask1_pth = "/content/drive/MyDrive/3D_Vision_Class/term_project/part1/part1_images4/IMG_4614_label.png"
  img_mask2_pth = "/content/drive/MyDrive/3D_Vision_Class/term_project/part1/part1_images4/IMG_4615_label.png"

  # Read the images
  inputs = read_images(img_path1, img_path2, img_mask1_pth, img_mask2_pth)

  # Feature detection and matching. If want to use mask, set use_mask=True
  src_points, target_points = detect_and_match_features(inputs, use_mask=True)

  # Estimate homography by using Normalized DLT with RANSAC
  ransac_params = {"num_iter": 1000, "epsilon": 5, "threshold": 0.4, "num_pnts": 7}
  out = ransac(src_points, target_points, ransac_params)

  print("Estimated Homography: ", out["H"])
  print("-----------------------------------------------------------------------")
  print("Number of best inlier correspondences: ", len(out["best_inliers1"]))
  print("-----------------------------------------------------------------------")

  # Test the estimated homography
  inlier_points1 = np.array(out["best_inliers1"]).squeeze(1)
  inlier_points2 = np.array(out["best_inliers2"]).squeeze(1)
  p1 = np.expand_dims(inlier_points1[0], 0)
  p1_aug = np.column_stack([p1, np.ones((1,1))]).T
  p2 = np.expand_dims(inlier_points2[0], 0)
  p2_aug = np.column_stack([p2, np.ones((1,1))]).T
  print("Distance: ", np.linalg.norm(p2_aug - out["H"].dot(p1_aug))**2)



  #### NOTE: SHOULD I CHECK FOR COLLINEARITY OF 3 POINTS ????



  # Test the estimated homography by using the opencv function
  m, mask = cv2.findHomography(inlier_points1, inlier_points2, cv2.RANSAC, confidence=0.99, ransacReprojThreshold=10.0)
  print("Opencv homography func distance: ", np.linalg.norm(p2_aug - m.dot(p1_aug)))
  #print("epsilon_H: ", np.linalg.norm(p2_aug - final_H.dot(p1_aug)))

  #img_matches = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1]+img2.shape[1], 3), dtype=np.uint8)
  #cv2.drawMatches(img1, kp1, img2, kp2, good_matches, img_matches, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
  #cv2_imshow(img_matches)
  #cv2.imwrite("./matches.png", img_matches)

  #kp_img = cv2.drawKeypoints(img1, kp1, None, color=(0, 255, 0), flags=0)
  #cv2_imshow(kp_img)

  # Image stitching
  img1 = inputs["img1"]
  img2 = inputs["img2"]
  panorama_creation(img1, img2, out["H"])

main()
